In [10]:
import coast
import xarray as xr
import matplotlib.pyplot as plt
import glob

In [2]:
import dask_gateway
# Create a connection to dask-gateway.
gw = dask_gateway.Gateway("https://dask-gateway.jasmin.ac.uk", auth="jupyterhub")

# Inspect and change the options if required before creating your cluster.
options = gw.cluster_options()
options.worker_cores = 1 #keeping this at 1 and allowing 15 worker processes seems to run faster than the other way around
options.scheduler_cores = 1 #we need at least one core for the scheduler
#specify which conda env to use, this must match the versions of python and dask (and a few other libraries) used on the notebook service
options.worker_setup='source /apps/jasmin/jaspy/mambaforge_envs/jaspy3.10/mf-22.11.1-4/bin/activate /gws/smf/j04/canari/dask-env'

# Create a dask cluster, or, if one already exists, connect to it.
# This stage creates the scheduler job in SLURM, so may take some time.
# While your job queues.
clusters = gw.list_clusters()
if not clusters:
    cluster = gw.new_cluster(options, shutdown_on_close=False)
else:
    cluster = gw.connect(clusters[0].name)

# Create at least one worker, and allow your cluster to scale to 15.
# The max JASMIN allows is 16, but one of these is used as the scheduler.
cluster.adapt(minimum=1, maximum=15)

# Get a dask client.
client = cluster.get_client()

ValueError: No JupyterHub API token provided, and JUPYTERHUB_API_TOKEN environment variable not found

In [ ]:
client

In [3]:
nemo_dom = "/gws/nopw/j04/canari/users/dlrhodso/mesh_mask.nc"

config_grid={}
config_dir="../../tutorials/config"
for grid in ['t','f','u','v']:
    config_grid[grid]=f'{config_dir}/example_nemo_grid_{grid}.json'

nemo_f = coast.Gridded( fn_domain=nemo_dom, config=config_grid['f'])

/home/users/eboland/.conda/envs/canari-dask-mine/lib/python3.10/site-packages/coast/data/gridded.py:236: UserWarning: The model domain loaded, '/gws/nopw/j04/canari/users/dlrhodso/mesh_mask.nc', does not contain the bathy_metry' variable. This will result in the NEMO.dataset.bathymetry variable being set to zero, which may result in unexpected behaviour from routines that require this variable.


In [24]:
Sref=34.8

In [ ]:
transect_name='GreenlandNorway'
transect_coords=((70,-22),(70,18))
tran_f=coast.TransectF(nemo_f,transect_coords[0],transect_coords[1])

data_path = "/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2"
ensembles=glob.glob(f'{data_path}/*')
for ens_file in ensembles:
#ens_file=ensembles[0]
    ens=int(ens_file.split('/')[-1])
    print(f'Ensemble {ens}')
    ens_data_path=f'{data_path}/{ens}/OCN/yearly'
    fout=f'../data/FWtransport_{transect_name}_{ens:02.0f}'
    if glob.glob(fout):
        print(f'File for {ens} exists, skipping')
        continue
    else:
        data_ufw_all=[]
        data_vfw_all=[]
        for year in range(2000,2015):
            year_data_path=f'{ens_data_path}/{year}'
            t_data_u = xr.open_mfdataset(f"{year_data_path}/*_mon__grid_U_vozocrtx.nc")
            t_data_us= xr.open_mfdataset(f"{year_data_path}/*_mon__grid_U_us.nc")
            t_data_e3u = xr.open_mfdataset(f"{year_data_path}/*_mon__grid_U_e3u.nc")
            ufw=(t_data_us.us-Sref*t_data_u.vozocrtx)/Sref
            ufw.name='uo'
            t_data_ufw=ufw.to_dataset()
            t_data_ufw['e3']=t_data_e3u.e3u
            data_ufw_all.append(t_data_ufw)
            
            t_data_v = xr.open_mfdataset(f"{year_data_path}/*_mon__grid_V_vomecrty.nc")
            t_data_vs= xr.open_mfdataset(f"{year_data_path}/*_mon__grid_V_vs.nc")
            t_data_e3v = xr.open_mfdataset(f"{year_data_path}/*_mon__grid_V_e3v.nc")
            vfw=(t_data_vs.vs-Sref*t_data_v.vomecrty)/Sref
            vfw.name='vo'
            t_data_vfw=vfw.to_dataset()
            t_data_vfw['e3']=t_data_e3v.e3v
            data_vfw_all.append(t_data_vfw)
            
        nemo_v = coast.Gridded(fn_data=xr.concat(data_vfw_all,'time_counter'), fn_domain=nemo_dom, config=config_grid['v'])
        nemo_u = coast.Gridded(fn_data=xr.concat(data_ufw_all,'time_counter'), fn_domain=nemo_dom, config=config_grid['u'])
        tran_f.calc_flow_across_transect(nemo_u, nemo_v)
        transports=tran_f.data_cross_tran_flow.normal_transports.sum('r_dim')
        transports.to_netcdf(fout)

Ensemble 1
File for 1 exists, skipping
Ensemble 10
File for 10 exists, skipping
Ensemble 11


/home/users/eboland/.conda/envs/canari-dask-mine/lib/python3.10/site-packages/coast/data/gridded.py:236: UserWarning: The model domain loaded, '/gws/nopw/j04/canari/users/dlrhodso/mesh_mask.nc', does not contain the bathy_metry' variable. This will result in the NEMO.dataset.bathymetry variable being set to zero, which may result in unexpected behaviour from routines that require this variable.
/home/users/eboland/.conda/envs/canari-dask-mine/lib/python3.10/site-packages/coast/data/gridded.py:236: UserWarning: The model domain loaded, '/gws/nopw/j04/canari/users/dlrhodso/mesh_mask.nc', does not contain the bathy_metry' variable. This will result in the NEMO.dataset.bathymetry variable being set to zero, which may result in unexpected behaviour from routines that require this variable.
